In [ ]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.optimizers import Adam

# Data loading
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    '/home/dragon/repos/datasets/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
    '/home/dragon/repos/datasets/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Model building
base_model = MobileNetV2(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)

# Save the model
model.save('skin_disease_detection_model.h5')


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Data loading
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
train_generator = train_datagen.flow_from_directory(
    '/home/dragon/repos/datasets/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
    '/home/dragon/repos/datasets/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Load pre-trained ResNet50 model without the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False)

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer with 256 hidden units and ReLU activation
x = Dense(256, activation='relu')(x)

# Add a final classification layer with softmax activation
predictions = Dense(9, activation='softmax')(x)

# Combine the base model and the new classification layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model (only train the new classification layers)
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)

# Evaluate the model
loss, accuracy = model.evaluate(val_generator, steps=val_generator.samples // val_generator.batch_size)
print(f'Validation loss: {loss:.4f}')
print(f'Validation accuracy: {accuracy:.4f}')


In [ ]:
import os
import numpy as np
from PIL import Image
from flask import Flask, request, jsonify, render_template
from tensorflow import keras

app = Flask(__name__)
model = keras.models.load_model('skin_disease_detection_model.h5')
target_size = (224, 224)  # Adjust based on your model's input shape

def preprocess_image(image):
    image = image.resize(target_size)
    image = np.array(image) / 255.0  # Normalize pixel values
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

def predict_image(image):
    preprocessed_image = preprocess_image(image)
    predictions = model.predict(preprocessed_image)
    predicted_class = np.argmax(predictions, axis=-1)
    return predicted_class

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload_image():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if file:
        image = Image.open(file)
        predicted_class = predict_image(image)
        return render_template('index.html', predicted_class=int(predicted_class))

if __name__ == '__main__':
    app.run(debug=True)


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision import models
import os

# Set device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transforms for data preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define paths to train and test data
train_data_path = '/home/oem/Downloads/whatsapp/archive/DATA/train'
test_data_path = '/home/oem/Downloads/whatsapp/archive/DATA/testing'

# Load train and test datasets
train_dataset = datasets.ImageFolder(train_data_path, transform=transform)
test_dataset = datasets.ImageFolder(test_data_path, transform=transform)

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Define CNN model architecture
class CustomModel(nn.Module):
    def __init__(self, num_classes):
        super(CustomModel, self).__init__()
        self.model = models.resnet18(pretrained=True)  # You can choose a different pretrained model if needed
        in_features = self.model.fc.in_features
        self.model.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.model(x)

# Initialize the model
num_classes = len(train_dataset.classes)
model = CustomModel(num_classes).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {(100 * correct / total):.2f}%")

# Save the trained model
torch.save(model.state_dict(), 'model.pt')


Epoch 1/10, Loss: 1.2791917281491416
Epoch 2/10, Loss: 0.7135993646723884
Epoch 3/10, Loss: 0.4265538305044174
Epoch 4/10, Loss: 0.3351859711110592
Epoch 5/10, Loss: 0.19149909647447722
Epoch 6/10, Loss: 0.1682263586137976
Epoch 7/10, Loss: 0.09487607622785228
Epoch 8/10, Loss: 0.07281158146049295
Epoch 9/10, Loss: 0.11417909039716635
Epoch 10/10, Loss: 0.14924550987780094
Accuracy on test set: 69.73%


In [16]:
##################33
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.models import resnet18

# Set device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transforms for data preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define paths to train and test data
train_data_path = '/home/oem/Downloads/whatsapp/archive/DATA/train'
test_data_path = '/home/oem/Downloads/whatsapp/archive/DATA/testing'

# Load train and test datasets
train_dataset = datasets.ImageFolder(train_data_path, transform=transform)
test_dataset = datasets.ImageFolder(test_data_path, transform=transform)

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

# Define CNN model architecture
class CustomModel(nn.Module):
    def __init__(self, num_classes):
        super(CustomModel, self).__init__()
        self.model = resnet18(pretrained=True)  # Load pre-trained ResNet18 model
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.model(x)

# Initialize the model
num_classes = len(train_dataset.classes)
model = CustomModel(num_classes).to(device)

# Load the trained model weights
MODEL_PATH = "model.pt"
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

# Define loss function and optimizer (optional if not training further)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# No need to train the model further if using pre-trained weights

# Evaluate the model on the test set
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {(100 * correct / total):.2f}%")


Accuracy on test set: 69.73%


In [21]:
from flask import Flask, render_template, request, redirect, url_for, send_file
from werkzeug.utils import secure_filename
from ultralytics import YOLO
import os
import json

app = Flask(__name__)

UPLOAD_FOLDER = 'uploads'
RESULT_FOLDER = '/home/oem/repos/MachineLEarning-School/face detection/skin blemishes/results'
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif'}
MODEL_PATH = '/home/oem/repos/MachineLEarning-School/face detection/skin blemishes/best.pt'

# Load the pre-trained YOLOv5 model
model = YOLO(MODEL_PATH)

def inference(image_path, result_folder):
    try:
        # Run inference on a single image
        results = model(image_path)
        result_data = []

        for i, result in enumerate(results):
            result_image_path = os.path.join(result_folder, f'result_{i}.jpg')
            result.save(filename=result_image_path)
            result_info = {
                "image_path": result_image_path,
                "detections": []
            }

            # Example: Extracting detected diseases from the model
            detected_diseases = []  # Replace this with the actual detected disease information
            for det in result.pred:
                class_label = model.names[int(det[-1])]
                detected_diseases.append(class_label)
            
            # Append detected diseases to result_info
            result_info["detected_diseases"] = detected_diseases

            for det in result.pred:
                # Extract class label and confidence
                class_label = model.names[int(det[-1])]
                confidence = det[-2]

                # Store detection information
                detection_info = {
                    "class": class_label,
                    "confidence": confidence,
                    "box": det[:4].tolist()  # Convert to list to serialize in JSON
                }
                result_info["detections"].append(detection_info)

            result_data.append(result_info)

        # Serialize the result data to JSON
        json_data = json.dumps(result_data, indent=4)
        with open(os.path.join(result_folder, 'result.json'), 'w') as json_file:
            json_file.write(json_data)

    except Exception as e:
        print(f"Error during inference: {e}")

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload():
    if 'file' not in request.files:
        return redirect(url_for('index'))

    file = request.files['file']

    if file.filename == '':
        return redirect(url_for('index'))

    if file and allowed_file(file.filename):
        try:
            filename = secure_filename(file.filename)
            file_path = os.path.join(UPLOAD_FOLDER, filename)
            file.save(file_path)

            # Perform inference on the uploaded image
            inference(file_path, RESULT_FOLDER)

            # Redirect to the result page
            return redirect(url_for('result'))
        except Exception as e:
            print(f"Error during file upload or inference: {e}")
            return redirect(url_for('index'))

    return redirect(url_for('index'))

@app.route('/result')
def result():
    return render_template('result.html')

@app.route('/display_result')
def display_result():
    result_image_path = os.path.join(RESULT_FOLDER, 'result_0.jpg')
    return send_file(result_image_path, mimetype='image/jpeg')

if __name__ == '__main__':
    os.makedirs(UPLOAD_FOLDER, exist_ok=True)
    os.makedirs(RESULT_FOLDER, exist_ok=True)
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/oem/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/oem/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/oem/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/home/oem/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/home/oem/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 331, in init_sockets
    self.shell_port = sel

SystemExit: 1

/home/oem/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
